# Embedding

This notebook evaluates methods for embedded document representation using the [academia.stackexchange.com](https://academia.stackexchange.com/) data dump.

## Table of Contents
* [Data import](#data_import)
* [Embedding](#embedding)
* [Experiments](#experiments)
* [Evaluation](#evaluation)
* [Dimension Reduction](#dim_reduce)

In [1]:
%load_ext autoreload
%autoreload 2

<a id='data_import'/>

## Data Import

In [2]:
from academia_tag_recommender.experiments.data import ExperimentalData

ed = ExperimentalData.load()

<a id='prep'/>

## Data Preparation

In [3]:
X_train, _, _, _ = ed.get_train_test_set()
X_train = [[x] for x in X_train]

<a id='embedding'/>

## Embedding

> Word embedding is a term used for the representation of words for text analysis, typically in the form of a real-valued vector that encodes the meaning of the word such that the words that are closer in the vector space are expected to be similar in meaning. [Wikipedia contributors (2021)][1]

The following word embedding models will be for this approach:
- Word2Vec
- Doc2Vec
- FastText


**Word2Vec**
> The word2vec tool takes a text corpus as input and produces the word vectors as output. It first constructs a vocabulary from the training text data and then learns vector representation of words. The resulting word vector file can be used as features in many natural language processing and machine learning applications. [Google (2013)][2]


**Doc2Vec**
> Le and Mikolov in 2014 introduced the Doc2Vec algorithm, which usually outperforms such simple-averaging of Word2Vec vectors. The basic idea is: act as if a document has another floating word-like vector, which contributes to all training predictions, and is updated like other word-vectors, but we will call it a doc-vector. [Radim Řehůřek (2020)][3]


**FastText**
> The main principle behind [F]astText is that the morphological structure of a word carries important information about the meaning of the word. Such structure is not taken into account by traditional word embeddings like Word2Vec, which train a unique word embedding for every individual word. [F]astText attempts to solve this by treating each word as the aggregation of its subwords. For the sake of simplicity and language-independence, subwords are taken to be the character ngrams of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams. [Radim Řehůřek (2020)][4]

[1]: https://en.wikipedia.org/wiki/Word_embedding
[2]: https://code.google.com/archive/p/word2vec/
[3]: https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py
[4]: https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py

<a id='experiments'/>

## Experiments

Before preprocessing a document is still a whole sentence, including punctuation and html tags.

In [4]:
print(X_train[0])

['What kind of Visa is required to work in Academia in Japan? <p>As from title. What kind of visa class do I have to apply for, in order to work as an academic in Japan ? </p>\n']


**Word2Vec**

First the string sentences are tokenized into arrays of strings representing the words. Punctuation and html tags are removed.

In [5]:
from academia_tag_recommender.embedded_data import Word2Tok
sentences = Word2Tok(X_train, flat=False)

print(list(sentences)[0:3])

[['kind', 'visa', 'required', 'work', 'academia', 'japan'], ['title'], ['kind', 'visa', 'class', 'apply', 'order', 'work', 'academic', 'japan']]


A `Word2Vec` model is trained using the tokenized sentences.

In [6]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=sentences)
wv = model.wv
del model
wv.init_sims(replace=True)
print('Training the model on the documents results in {} words in the vocabulary.'.format(len(wv.vocab)))

Training the model on the documents results in 12816 words in the vocabulary.


The `Word2Vec` can now be used to generate a vectors for words. Per default implementation the vector has 100 features.

In [7]:
wv['academic']

array([-0.01126248, -0.03342965,  0.10469431,  0.07857093, -0.03391938,
        0.00404332,  0.10155719,  0.07995059, -0.08427179,  0.1122121 ,
       -0.07148434, -0.08021346, -0.0962241 ,  0.00917253, -0.0475006 ,
        0.1242073 ,  0.14381252, -0.0881118 ,  0.16303246, -0.11870279,
       -0.19611287,  0.064424  ,  0.11835656,  0.06517876, -0.10907665,
        0.02775248, -0.06966034,  0.05161316, -0.00509942,  0.02610457,
       -0.02165228, -0.07894783,  0.05706049,  0.02305102, -0.08014485,
        0.22982506, -0.09516134, -0.00195995, -0.05212987, -0.10273121,
       -0.08041546,  0.04897661,  0.03315542, -0.02326958,  0.1052165 ,
       -0.11830541, -0.03169212,  0.03952116, -0.12340844,  0.10725659,
        0.08625586,  0.08291639,  0.11055885,  0.04388536, -0.05895904,
       -0.02768142, -0.13496156, -0.13655935, -0.10573473, -0.03899273,
       -0.05418871, -0.07987771, -0.11436221, -0.02675861,  0.2436887 ,
        0.02814967,  0.06634986, -0.07372411, -0.03123233, -0.06

Based on these vectors words can be compared to each other. The 10 most similar words to `academic` are:

In [8]:
wv.most_similar('academic')

[('professional', 0.5941072702407837),
 ('scientific', 0.5231539011001587),
 ('existent', 0.49167492985725403),
 ('academia', 0.4735167622566223),
 ('traditional', 0.4654197692871094),
 ('prospects', 0.4418567419052124),
 ('academics', 0.4371306896209717),
 ('educational', 0.4251668453216553),
 ('socioeconomic', 0.4145768880844116),
 ('advancement', 0.4130385220050812)]

**Combining Word2Vec with multiword n-grams**

Instead of only using unigrams it is possible to include bigrams into the vocabulary.

In [9]:
from academia_tag_recommender.embedded_data import Word2Tok
sentences = Word2Tok(X_train, flat=False)

print(list(sentences)[0:3])

[['kind', 'visa', 'required', 'work', 'academia', 'japan'], ['title'], ['kind', 'visa', 'class', 'apply', 'order', 'work', 'academic', 'japan']]


In [10]:
from gensim.models.phrases import Phrases

bigram_transformer = Phrases(sentences, min_count=1)

In [11]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=bigram_transformer[sentences])
wv = model.wv
del model
wv.init_sims(replace=True)
print('Training the model on the documents results in {} words in the vocabulary.'.format(len(wv.vocab)))

Training the model on the documents results in 20473 words in the vocabulary.


Including bigrams the vocabulary increases nearly by factor 2.

In [12]:
wv['academic']

array([ 0.0495455 , -0.00419593,  0.11066435, -0.02374866, -0.22952871,
       -0.08000941,  0.02771649,  0.01024842, -0.08156473,  0.12846893,
       -0.04128965, -0.07169883, -0.09920597, -0.13022473,  0.03891062,
        0.00903767,  0.20555912, -0.0070736 ,  0.04888603,  0.05045787,
       -0.1317934 ,  0.07066325, -0.02096056,  0.04510014,  0.0181801 ,
        0.05664595, -0.11644613,  0.12939735, -0.00432711, -0.08152245,
       -0.18457815, -0.2119369 ,  0.09498297, -0.07877592, -0.06825345,
        0.14617655,  0.09528144,  0.05029193,  0.04541614,  0.04639877,
       -0.01172495,  0.15301225, -0.07834055,  0.01008555, -0.13247669,
       -0.13601477, -0.03733093, -0.15711571, -0.09747767,  0.02808852,
        0.06926011,  0.02506783,  0.15970664,  0.03217129, -0.09766825,
        0.00457934, -0.16741234,  0.00227875, -0.11511894, -0.14892387,
       -0.04949791,  0.03258445, -0.13882124, -0.05990782, -0.04662485,
        0.04948926,  0.02993926,  0.06069308, -0.00270678, -0.08

In [13]:
wv.most_similar('academic')

[('professional', 0.6862949728965759),
 ('non_academic', 0.6769979000091553),
 ('tt_faculty', 0.6094162464141846),
 ('terms', 0.6052592992782593),
 ('teaching_experience', 0.6019759774208069),
 ('early_career', 0.5860946774482727),
 ('blended', 0.5758219361305237),
 ('permanent', 0.5706329345703125),
 ('industrial', 0.5703277587890625),
 ('benefits', 0.5627210736274719)]

Looking at the simililarity of words, there are now many bigrams that are similar to `academic`.

**Doc2Vec**

The `Doc2Vec` trains the model supervised, using the labels. Therefore all documents first need to be tokenized and connected to their label.

In [14]:
from academia_tag_recommender.embedded_data import Doc2Tagged

tokens = Doc2Tagged(X_train, tag=True)

token_list = list(tokens)
token_list[0]

TaggedDocument(words=['kind', 'visa', 'required', 'work', 'academia', 'japan', 'title', 'kind', 'visa', 'class', 'apply', 'order', 'work', 'academic', 'japan'], tags=[0])

In [15]:
from gensim.models import Doc2Vec
model = Doc2Vec()
model.build_vocab(token_list)
model.train(token_list, total_examples=model.corpus_count, epochs=model.epochs)
print('Training the model on the documents results in {} words out of {} documents in the vocabulary.'.format(len(model.wv.vocab), len(model.docvecs)))

Training the model on the documents results in 12816 words out of 24812 documents in the vocabulary.


In [16]:
vector = model.infer_vector(['academic'])
vector

array([-0.01246654,  0.01008995,  0.01704104, -0.026956  ,  0.00418499,
        0.03721834, -0.01852683,  0.00209496, -0.01860138,  0.03959585,
        0.00507056,  0.02848887, -0.01308582, -0.01310561, -0.00747064,
       -0.01693214,  0.00857303, -0.00355638,  0.02803446,  0.0198663 ,
       -0.00133561,  0.01337443, -0.01685062,  0.00772604,  0.0478564 ,
       -0.02584357,  0.02873253,  0.01621348, -0.00468331, -0.01382237,
       -0.01595827, -0.05603697,  0.01256992,  0.00667518, -0.00173867,
        0.01282012,  0.04187428,  0.00897409,  0.04029883, -0.03409013,
        0.01656222, -0.05447659, -0.02043061,  0.01358684,  0.01051033,
       -0.01498997,  0.004082  , -0.00109008, -0.01355278,  0.01846856,
       -0.00381788, -0.03398077, -0.02348213, -0.02855602, -0.04648814,
       -0.00220863, -0.06394363,  0.00079038, -0.05311364, -0.01619767,
       -0.02317375, -0.02261864, -0.02486599,  0.02838418, -0.00781799,
        0.05829245, -0.02244406,  0.00849184, -0.02140814,  0.00

With the `Doc2Vec` model one can examine documents similar to the keyword `academic` instead of similar words.

In [17]:
similar_docs = model.docvecs.most_similar([vector])
print(similar_docs[0], token_list[similar_docs[0][0]].words)
print(similar_docs[1], token_list[similar_docs[1][0]].words)

(17992, 0.8779444694519043) ['academic', 'prizes', 'awards', 'vs', 'academic', 'achievements', 'phd', 'scholarship', 'application', 'forms', 'mean', 'details', 'academic', 'prizes', 'awards', 'details', 'academic', 'achievements', 'academic', 'prizes', 'academic', 'achievements', 'things']
(1594, 0.8731231093406677) ['academic', 'statement', 'purpose', 'vs', 'nsf', 'personal', 'statement', 'beyond', 'obvious', 'nsf', 'fellowship', 'personal', 'statement', 'good', 'nsf', 'academic', 'statement', 'purpose', 'graduate', 'school', 'application', 'grad', 'school', 'grad', 'school', 'different', 'similar', 'two', 'essays', 'already', 'written', 'nsf', 'fellowship', 'application', 'now', 'process', 'writing', 'academic', 'statement', 'purpose', 'grad', 'school', 'applications', 'trying', 'figure', 'similar', 'statements', 'going', 'end']


In [18]:
wv = model.wv
del model
wv.init_sims(replace=True)
wv.most_similar('academic')

[('scientific', 0.5339624285697937),
 ('professional', 0.5270957946777344),
 ('academia', 0.45144784450531006),
 ('existent', 0.4350164830684662),
 ('traditional', 0.43040335178375244),
 ('academics', 0.4162779450416565),
 ('future', 0.4070352613925934),
 ('market', 0.3943069875240326),
 ('prospects', 0.38636696338653564),
 ('tt', 0.3860465884208679)]

**FastText**

In [19]:
from academia_tag_recommender.embedded_data import Word2Tok
sentences = Word2Tok(X_train, flat=False)

print(list(sentences)[0:3])

[['kind', 'visa', 'required', 'work', 'academia', 'japan'], ['title'], ['kind', 'visa', 'class', 'apply', 'order', 'work', 'academic', 'japan']]


In [20]:
from gensim.models import FastText
model = FastText(window=3, min_count=2)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=model.corpus_count, epochs=20)
wv = model.wv
del model
wv.init_sims(replace=True)
print('Training the model on the documents results in {} words in the vocabulary.'.format(len(wv.vocab)))

Training the model on the documents results in 22164 words in the vocabulary.


In [21]:
wv['academic']

array([-0.01360371,  0.2201108 , -0.07534797, -0.02042685,  0.06911746,
       -0.08047682,  0.00112343, -0.00682863, -0.00313658, -0.06221205,
        0.11683771,  0.09753276, -0.11210427,  0.02122977, -0.08609265,
       -0.00085879,  0.0962998 , -0.16790262,  0.01899188,  0.0365886 ,
       -0.01357992,  0.0850454 , -0.12691697, -0.12235729, -0.22409484,
       -0.0094045 , -0.06140542,  0.09440521,  0.00361523,  0.24553874,
       -0.03838311, -0.08878055, -0.14826025, -0.15915938, -0.1602885 ,
       -0.03411989, -0.10631298, -0.14225276,  0.04898833,  0.02420743,
        0.06041806, -0.10884795, -0.02548579,  0.1116297 , -0.01417138,
        0.07347418,  0.03819118,  0.05620288,  0.05335264, -0.00278359,
        0.00099527, -0.05098832, -0.05446682,  0.25842884,  0.00167618,
        0.03151484, -0.05252331, -0.01290049,  0.05672811,  0.14226958,
       -0.03228727, -0.13016078,  0.19038355,  0.01075957, -0.09900304,
       -0.08866976, -0.10331886,  0.0975347 , -0.02733367,  0.11

In [22]:
wv.most_similar('academic')

[('nonacademic', 0.937012791633606),
 ('unacademic', 0.9337728023529053),
 ('academica', 0.9190360307693481),
 ('academico', 0.9158546924591064),
 ('academical', 0.8972895741462708),
 ('academy', 0.8572608232498169),
 ('academe', 0.8453583717346191),
 ('academies', 0.8380265831947327),
 ('academician', 0.8120579719543457),
 ('acad', 0.8003551959991455)]

Since `FastText` uses character n-grams there are now many words very close to `academia`.

**Combining FastText with multiword n-grams**

`FastText` can be extended with bigrams too.

In [23]:
from academia_tag_recommender.embedded_data import Word2Tok
sentences = Word2Tok(X_train, flat=False)

print(list(sentences)[0:3])

[['kind', 'visa', 'required', 'work', 'academia', 'japan'], ['title'], ['kind', 'visa', 'class', 'apply', 'order', 'work', 'academic', 'japan']]


In [24]:
from gensim.models.phrases import Phrases

bigram_transformer = Phrases(sentences, min_count=1)

In [25]:
from gensim.models import FastText
model = FastText(window=3, min_count=2)
model.build_vocab(sentences=bigram_transformer[sentences])
model.train(sentences=sentences, total_examples=model.corpus_count, epochs=20)
wv = model.wv
del model
wv.init_sims(replace=True)
print('Training the model on the documents results in {} words in the vocabulary.'.format(len(wv.vocab)))

Training the model on the documents results in 55899 words in the vocabulary.


In [26]:
wv['academic']

array([-5.03340252e-02, -1.59247424e-02, -4.10016216e-02,  7.63774291e-02,
        4.06169519e-03, -1.35998011e-01, -7.71520957e-02, -6.16722852e-02,
       -2.27796678e-02,  1.22583494e-01, -1.33953750e-01, -1.96712557e-02,
       -4.90332432e-02,  1.07224926e-01,  1.46762058e-01, -5.59856370e-02,
       -5.77349355e-03, -2.17589810e-02, -1.48361757e-01,  5.79931699e-02,
        4.38637054e-03,  9.82263684e-02, -1.17381424e-01,  1.74733922e-01,
       -6.09970503e-02, -4.47782464e-02,  2.15554163e-01, -4.36310768e-02,
       -5.42981327e-02,  3.17662358e-02,  8.23094696e-03,  3.18498164e-02,
        7.35429674e-02, -5.03120981e-02, -5.03785610e-02, -2.83606231e-01,
        6.30341917e-02,  1.89638585e-01,  1.05324514e-01, -9.09575373e-02,
       -1.92452475e-01,  5.21142855e-02,  2.62836479e-02, -2.74904091e-02,
       -1.62085980e-01,  8.32487047e-02,  1.52187854e-01, -6.84334561e-02,
       -1.04442351e-01, -1.52494133e-01, -4.32568304e-02,  4.41923924e-03,
       -1.05814986e-01, -

In [27]:
wv.most_similar('academic')

[('vp_academic', 0.991126537322998),
 ('climb_academic', 0.9646323919296265),
 ('non_academic', 0.960191011428833),
 ('climbs_academic', 0.9601061940193176),
 ('wear_academic', 0.9575188755989075),
 ('academic_cvs', 0.9546220898628235),
 ('nonacademic', 0.9523471593856812),
 ('unacademic', 0.9492830038070679),
 ('academic_theft', 0.9459648132324219),
 ('academica', 0.9448950886726379)]

Similar words to `academic` are now bigrams like `non academic` and unigrams like `nonacademic`.